<span STYLE="font-size:200%"> 
    Data analysis of the BOSE stress-relaxation test
</span>

Docker image: gnasello/datascience-env:2023-07-16 \
Conda environment: R \
Latest update: 26 July 2023

# Load required packages and data

In [ ]:
library(ggplot2)
library(ggpubr)
library(investr)

Load the [r_utils scirpts](https://github.com/gabnasello/r_utils.git) from GitHub

You can debug the *r_utils* script by downloading the GitHub project from terminal and chance the argument of the `source` function (open it from JupyterLab):

`git clone https://github.com/gabnasello/r_utils.git`

How to Reuse Functions That You Create In Scripts, [tutorial](https://www.earthdatascience.org/courses/earth-analytics/multispectral-remote-sensing-data/source-function-in-R/)

In [ ]:
source("https://raw.githubusercontent.com/gabnasello/r_utils/main/ggplot_utils.R")
source("https://raw.githubusercontent.com/gabnasello/r_utils/main/stats_utils.R")

Set the parameters of the `cyto_plot()` function

In [ ]:
label_text_size = 1.4; label_fill_alpha = 0.2; label_text_font = 1
axes_text_size = 1.4
axes_label_text_size = 1.6; title_text_size = 1.5; header_text_size = 1.3
legend_text_size = 1.7

## Load data

<span style="color:red">**User Input**</span>

In [ ]:
filetable <- 'data/GN015_C-2_1 04062023 040128_tdf.CSV'

In [ ]:
# skip - the first three lines of tdf.CSV files are not relevant for the dataframe object 
df_original <- read.csv(file=filetable, skip = 3)
head(df_original)

Show units

In [ ]:
units <- df_original[1,]
units

# Data pre-processing

## Remove not relevant data

In [ ]:
#delete 1st row
df <- df_original[-1,]

# Remove DataFrame last column with Base R
df <- df[1: ncol(df)-1 ]

# Convert all variable types to numeric
df <- as.data.frame(apply(df, 2, as.numeric))  
head(df)

## Create time variable

In [ ]:
df['time'] <- df['Elapsed.Time']
head(df)

## Create load variable

Set the initial load value to zero, which corresponds to the tare load. 

In [ ]:
df['load'] <- df['Load.3'] - max(df[1:20, 'Load.3'])
head(df)

## Create displacement variable

Set the initial position of the plate to zero

In [ ]:
df['displacement'] = df[1, 'Disp'] - df['Disp']
head(df)

# Load and Displament vs Time

In [ ]:
p1 <- ggplot(df, aes(x = time, y = load)) + 
     geom_line(linewidth=1, color="#00A087FF")    

img1 <- ggplotMinAethetics(p1, width=7, height=4,
                          plot.title=element_text(size = 13),
                          xlabel='Time (s)', 
                          ylabel='Load (g)', 
                          legend.position="none"
                          )

p2 <- ggplot(df, aes(x = time, y = Disp)) + 
     geom_line(linewidth=1, color="#3C5488FF")    

img2 <- ggplotMinAethetics(p2, width=7, height=4,
                          plot.title=element_text(size = 13),
                          xlabel='Time (s)', 
                          ylabel='Displacement (mm)', 
                          legend.position="none"
                          )

img1
img2

# Select interval of interest

<span style="color:red">**User Input**</span>

In [ ]:
test_start <- 2 #seconds

In [ ]:
df <- df[df['time'] <= test_start, ]
head(df)

# Convert Load (in grams) to Force (in milli Newtons) and change sign

In [ ]:
g_acc = 9.80665 # m/s2

df['force'] = df['load'] * (-g_acc) # g * m/s^2 = mN (10^-3 N)
head(df)

# Raw Plots

## Load vs Time

In [ ]:
p <- ggplot(df, aes(x = time, y = load)) + 
     geom_line(linewidth=1, color="#00A087FF")    

img <- ggplotMinAethetics(p, width=3.5, height=4,
                          plot.title=element_text(size = 13),
                          xlabel='Time (s)', 
                          ylabel='Load (g)', 
                          legend.position="none"
                          )
       # scale_y_continuous(expand=c(0,0), limits=c(0,170)) 

img

## Force vs Displacement

In [ ]:
p <- ggplot(df, aes(x = displacement, y = force)) + 
     geom_point(size=1, color="#00A087FF")    

img <- ggplotMinAethetics(p, width=3.5, height=4,
                          plot.title=element_text(size = 13),
                          xlabel='Displacement (mm)', 
                          ylabel='Force (mN)', 
                          legend.position="none"
                          )

img

# Unit conversions

## Convert forces to stresses (Pa)

<span style="color:red">**User Input**</span>

In [ ]:
radius <- 5 # mm

sample_area <- pi * (radius^2) # mm^2

df['stress'] <- df['force']/sample_area * 1e03 # mN / mm^2 * 10**3 = Pa
head(df)

## Convert displacements to strains (%)

<span style="color:red">**User Input**</span>

In [ ]:
sample_thickness <- df[1,'NetDisp'] # mm

df['strain'] <- df['displacement']/sample_thickness #
df['strain100'] <- df['strain'] * 100
head(df)

# Stress vs Strain plot

In [ ]:
p <- ggplot(df, aes(x = strain100, y = stress)) + 
     geom_point(size=1, color="#00A087FF")    

img <- ggplotMinAethetics(p, width=3.5, height=4,
                          plot.title=element_text(size = 13),
                          xlabel='Compressive Strain (%)', 
                          ylabel='Stress (Pa)', 
                          legend.position="none"
                          )

img

# Save data

Save new `.csv` file in the same directory where the orginal dataset is located

In [ ]:
filename_stem <- tools::file_path_sans_ext(filetable)
newfile <- paste(filename_stem, '_compression_analyzed.csv')
newfile

In [ ]:
write.csv(df, newfile)

# Young modulus (E) calculation

In [ ]:
young_values <- data.frame(min_strain100=double(),
                           max_strain100=double(),
                           young_pa=double()
                           )
young_values

## E 0-1% strain

In [ ]:
min_strain100 = 0
max_strain100 = 1

In [ ]:
df_young <- df[df['strain100']>=min_strain100 & df['strain100']<=max_strain100,]
tail(df_young)

### Build regression model

In [ ]:
model = lm( stress ~ strain + 0, data=df_young)
summary(model)

In [ ]:
# young modulus in Pascal
young_pa <- unname(model$coefficients['strain'])

message <- sprintf('--------------\n\n\nYoung modulus: %.2f Pa\n\n\n--------------', young_pa)
cat(message)

In [ ]:
young_values <- rbind(young_values, data.frame(min_strain100=min_strain100,
                                               max_strain100=max_strain100,
                                               young_pa=young_pa
                                               )
                     )
young_values

### Plot regression model

Add 'fit', 'lwr', and 'upr' columns to dataframe (generated by predict). This is useful to [to plot an already-existing linear model](https://stackoverflow.com/questions/44865508/using-ggplot2-to-plot-an-already-existing-linear-model)

In [ ]:
model.predict <- cbind(df, predict(model, df['strain'], interval = 'confidence'))
head(model.predict)

plot the points (actual observations), regression line, and confidence interval

In [ ]:
p <- ggplot(model.predict, aes(x = strain100, y = stress)) + 
     geom_point(size=1, color="#4DBBD5FF", alpha=0.8)  + 
     geom_line(aes(strain100, fit))

img <- ggplotMinAethetics(p, width=3.5, height=4,
                          plot.title=element_text(size = 13),
                          xlabel='Compressive Strain (%)', 
                          ylabel='Stress (Pa)', 
                          legend.position="none"
                          )
       # scale_y_continuous(expand=c(0,0), limits=c(0,260)) 

img

## E 0-5% strain

In [ ]:
min_strain100 = 0
max_strain100 = 5

In [ ]:
df_young <- df[df['strain100']>=min_strain100 & df['strain100']<=max_strain100,]
tail(df_young)

### Build regression model

In [ ]:
model = lm( stress ~ strain + 0, data=df_young)
summary(model)

In [ ]:
# young modulus in Pascal
young_pa <- unname(model$coefficients['strain'])

message <- sprintf('--------------\n\n\nYoung modulus: %.2f Pa\n\n\n--------------', young_pa)
cat(message)

In [ ]:
young_values <- rbind(young_values, data.frame(min_strain100=min_strain100,
                                                 max_strain100=max_strain100,
                                                 young_pa=young_pa
                                                 )
                     )
young_values

### Plot regression model

Add 'fit', 'lwr', and 'upr' columns to dataframe (generated by predict). This is useful to [to plot an already-existing linear model](https://stackoverflow.com/questions/44865508/using-ggplot2-to-plot-an-already-existing-linear-model)

In [ ]:
model.predict <- cbind(df, predict(model, df['strain'], interval = 'confidence'))
head(model.predict)

plot the points (actual observations), regression line, and confidence interval

In [ ]:
p <- ggplot(model.predict, aes(x = strain100, y = stress)) + 
     geom_point(size=1, color="#4DBBD5FF", alpha=0.8)  + 
     geom_line(aes(strain100, fit))

img <- ggplotMinAethetics(p, width=3.5, height=4,
                          plot.title=element_text(size = 13),
                          xlabel='Compressive Strain (%)', 
                          ylabel='Stress (Pa)', 
                          legend.position="none"
                          )
       # scale_y_continuous(expand=c(0,0), limits=c(0,260)) 

img

## E 0-10% strain

In [ ]:
min_strain100 = 0
max_strain100 = 10

In [ ]:
df_young <- df[df['strain100']>=min_strain100 & df['strain100']<=max_strain100,]
tail(df_young)

### Build regression model

In [ ]:
model = lm( stress ~ strain + 0, data=df_young)
summary(model)

In [ ]:
# young modulus in Pascal
young_pa <- unname(model$coefficients['strain'])

message <- sprintf('--------------\n\n\nYoung modulus: %.2f Pa\n\n\n--------------', young_pa)
cat(message)

In [ ]:
young_values <- rbind(young_values, data.frame(min_strain100=min_strain100,
                                                 max_strain100=max_strain100,
                                                 young_pa=young_pa
                                                 )
                     )
young_values

### Plot regression model

Add 'fit', 'lwr', and 'upr' columns to dataframe (generated by predict). This is useful to [to plot an already-existing linear model](https://stackoverflow.com/questions/44865508/using-ggplot2-to-plot-an-already-existing-linear-model)

In [ ]:
model.predict <- cbind(df, predict(model, df['strain'], interval = 'confidence'))
head(model.predict)

plot the points (actual observations), regression line, and confidence interval

In [ ]:
p <- ggplot(model.predict, aes(x = strain100, y = stress)) + 
     geom_point(size=1, color="#4DBBD5FF", alpha=0.8)  + 
     geom_line(aes(strain100, fit))

img <- ggplotMinAethetics(p, width=3.5, height=4,
                          plot.title=element_text(size = 13),
                          xlabel='Compressive Strain (%)', 
                          ylabel='Stress (Pa)', 
                          legend.position="none"
                          )
       # scale_y_continuous(expand=c(0,0), limits=c(0,130)) 

img

## Save data

In [ ]:
young_values

Save new `.csv` file in the same directory where the orginal dataset is located

In [ ]:
filename_stem <- tools::file_path_sans_ext(filetable)
newfile <- paste(filename_stem, '_young_modulus.csv')
newfile

In [ ]:
write.csv(young_values, newfile)